# 데이터셋 마운트 및 압축 해제

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive
!unzip cola_public_1.1.zip

/content/drive/MyDrive
Archive:  cola_public_1.1.zip
replace cola_public/README? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: cola_public/README      
replace cola_public/tokenized/in_domain_dev.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: cola_public/tokenized/in_domain_dev.tsv  
replace cola_public/tokenized/in_domain_train.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: cola_public/tokenized/in_domain_train.tsv  
replace cola_public/tokenized/out_of_domain_dev.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: cola_public/tokenized/out_of_domain_dev.tsv  
replace cola_public/raw/in_domain_dev.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: cola_public/raw/in_domain_dev.tsv  
replace cola_public/raw/in_domain_train.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace cola_public/raw/in_domain_train.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: cola_public/raw/in_domain_train.tsv  
  inflating: co

# 모델 다운로드

In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.8 MB/s 
     |████████████████████████████████| 182 kB 69.6 MB/s 
     |████████████████████████████████| 7.6 MB 57.9 MB/s 


In [4]:
from transformers import AutoConfig, AutoTokenizer, AutoModelForSequenceClassification

In [10]:
config = AutoConfig.from_pretrained('xlnet-base-cased')
tokenizer= AutoTokenizer.from_pretrained('xlnet-base-cased')
model= AutoModelForSequenceClassification.from_pretrained('xlnet-base-cased')

Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/467M [00:00<?, ?B/s]

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.weight', 'sequence_summary.summary.bias', 'logits_proj.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

# 라이브러리

In [5]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from keras_preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset,DataLoader,RandomSampler,SequentialSampler
from transformers import AdamW

In [6]:
if torch.cuda.is_available():
    device=torch.device("cuda")
else:
    devcie=torch.device("cpu")

In [7]:
device

device(type='cuda')

# data 구조 파악 및 전처리

In [12]:
%cd /content/drive/MyDrive
df = pd.read_csv("cola_public/raw/in_domain_train.tsv", sep="\t") # sep=\t를 추가해서 파일을 제대로 읽어오도록
df.head()

/content/drive/MyDrive


,gj04,1,Unnamed: 2,"Our friends won't buy this analysis, let alone the next one we propose."
0,gj04,1,NaN,One more pseudo generalization and I'm giving up.
1,gj04,1,NaN,One more pseudo generalization or I'm giving up.
2,gj04,1,NaN,"The more we study verbs, the crazier they get."
3,gj04,1,NaN,Day by day the facts are getting murkier.
4,gj04,1,NaN,I'll fix you a drink.


In [13]:
df = pd.read_csv("./cola_public/raw/in_domain_train.tsv", delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])
df.head()

,sentence_source,label,label_notes,sentence
0,gj04,1,NaN,"Our friends won't buy this analysis, let alone..."
1,gj04,1,NaN,One more pseudo generalization and I'm giving up.
2,gj04,1,NaN,One more pseudo generalization or I'm giving up.
3,gj04,1,NaN,"The more we study verbs, the crazier they get."
4,gj04,1,NaN,Day by day the facts are getting murkier.


In [14]:
df=df[['sentence','label']]
df.head()

,sentence,label
0,"Our friends won't buy this analysis, let alone...",1
1,One more pseudo generalization and I'm giving up.,1
2,One more pseudo generalization or I'm giving up.,1
3,"The more we study verbs, the crazier they get.",1
4,Day by day the facts are getting murkier.,1


In [15]:
train_text, test_text, train_labels, test_labels = train_test_split(df['sentence'], df['label'], 
                                                                    test_size=0.2, 
                                                                    stratify=df['label'])

In [16]:
max_seq_len=64
tokens_train = tokenizer.batch_encode_plus(
    train_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

tokens_test = tokenizer.batch_encode_plus(
    test_text.tolist(),
    max_length = max_seq_len,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [17]:
train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
train_y = torch.tensor(train_labels.tolist())

test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
test_y = torch.tensor(test_labels.tolist())

In [18]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 64

train_data = TensorDataset(train_seq, train_mask, train_y)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

test_data = TensorDataset(test_seq, test_mask, test_y)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

# Fine Tune

In [19]:
from torch.cuda.amp import GradScaler

def train():
  
  model.train()

  total_loss, total_accuracy = 0, 0
  
  total_preds=[]
  
  for step,batch in enumerate(train_dataloader):
    
    if step % 50 == 0 and not step == 0:
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

    batch = [r.to(device) for r in batch]
 
    sent_id, mask, labels = batch
    inputs = {
        "input_ids": sent_id,
        "attention_mask": mask,
        "labels": labels
    }
    
    scaler = GradScaler(enabled=True) 
    model.zero_grad()        

    loss, logits = model(**inputs)[:2]
    preds=logits.argmax(1)
    
    scaler.scale(loss).backward()

    total_loss = total_loss + loss.item()

    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    optimizer.step()
    preds=preds.detach().cpu().numpy()

    total_preds.append(preds)

  avg_loss = total_loss / len(train_dataloader)
  total_preds  = np.concatenate(total_preds, axis=0)

  return avg_loss, total_preds

In [20]:
def evaluate():
  
  print("\nEvaluating...")
  
  model.eval()

  total_loss, total_accuracy = 0, 0
  
  total_preds = []


  for step,batch in enumerate(test_dataloader):
    
    if step % 50 == 0 and not step == 0:
      elapsed = format_time(time.time() - t0)
            
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(test_dataloader)))

    batch = [t.to(device) for t in batch]

    sent_id, mask, labels = batch
    inputs = {
        "input_ids": sent_id,
        "attention_mask": mask,
        "labels": labels
    }


    with torch.no_grad():       
        loss, logits = model(**inputs)[:2]
        preds=logits.argmax(1)

        total_loss = total_loss + loss.item()

        preds = preds.detach().cpu().numpy()

        total_preds.append(preds)

  avg_loss = total_loss / len(test_dataloader) 
  total_preds  = np.concatenate(total_preds, axis=0)

  return avg_loss, total_preds

In [22]:
model = model.to(device)
optimizer = AdamW(model.parameters(), lr = 1e-3)
epochs = 10

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [24]:
best_test_loss = float('inf')

train_losses=[]
test_losses=[]

for epoch in range(epochs):
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    
    train_loss, _ = train()
    test_loss, _ = evaluate()
    
    if test_loss < best_test_loss:
        best_valid_loss = test_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    
    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {test_loss:.3f}')


 Epoch 1 / 10
  Batch    50  of    107.
  Batch   100  of    107.

Evaluating...

Training Loss: 0.644
Validation Loss: 0.648

 Epoch 2 / 10
  Batch    50  of    107.
  Batch   100  of    107.

Evaluating...

Training Loss: 0.627
Validation Loss: 0.622

 Epoch 3 / 10
  Batch    50  of    107.
  Batch   100  of    107.

Evaluating...

Training Loss: 0.632
Validation Loss: 0.671

 Epoch 4 / 10
  Batch    50  of    107.
  Batch   100  of    107.

Evaluating...

Training Loss: 0.627
Validation Loss: 0.608

 Epoch 5 / 10
  Batch    50  of    107.
  Batch   100  of    107.

Evaluating...

Training Loss: 0.634
Validation Loss: 0.633

 Epoch 6 / 10
  Batch    50  of    107.
  Batch   100  of    107.

Evaluating...

Training Loss: 0.627
Validation Loss: 0.671

 Epoch 7 / 10
  Batch    50  of    107.
  Batch   100  of    107.

Evaluating...

Training Loss: 0.652
Validation Loss: 0.617

 Epoch 8 / 10
  Batch    50  of    107.
  Batch   100  of    107.

Evaluating...

Training Loss: 0.625
Validat